# How to view and update past graph state

Once you start [checkpointing](./persistence.ipynb) your graphs, you can easily **get** or **update** the state of the agent at any point in time. This permits a few things:

1. You can surface a state during an interrupt to a user to let them accept an action.
2. You can **rewind** the graph to reproduce or avoid issues.
3. You can **modify** the state to embed your agent into a larger system, or to let the user better control its actions.

The key methods used for this functionality are:

- [get_state](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.get_state): fetch the values from the target config
- [update_state](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.update_state): apply the given values to the target state

**Note:** this requires passing in a checkpointer.

Below is a quick example.

## Setup

First we need to install the packages required

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
_set_env("LANGCHAIN_API_KEY")

## Build the agent

We can now build the agent. We will build a relatively simple ReAct-style agent that does tool calling. We will use Anthropic's models and a fake tool (just for demo purposes).

In [1]:
# Set up the state
from langgraph.graph import MessagesState

# Set up the tool
from langchain_core.tools import tool
from langgraph.prebuilt import ToolExecutor


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return [
        f"I looked up: {query}. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    ]


tools = [search]
tool_executor = ToolExecutor(tools)

# Set up the model
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")
model = model.bind_tools(tools)

# Define nodes and conditional edges

from langchain_core.messages import ToolMessage

from langgraph.prebuilt import ToolInvocation


# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_call = last_message.tool_calls[0]
    action = ToolInvocation(
        tool=tool_call["name"],
        tool_input=tool_call["args"],
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a ToolMessage
    tool_message = ToolMessage(
        content=str(response), name=action.tool, tool_call_id=tool_call["id"]
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [tool_message]}


# Build the graph

from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Set up memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

/Users/harrisonchase/.pyenv/versions/3.11.1/envs/permchain/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatAnthropic.bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


## Interacting with the Agent

We can now interact with the agent. Let's ask it for the weather in SF.


In [4]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Use the search tool to look up the weather in SF")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Use the search tool to look up the weather in SF
================================== Ai Message ==================================

[{'text': "Certainly! I'll use the search tool to look up the weather in San Francisco for you. Let me do that right away.", 'type': 'text'}, {'id': 'toolu_01M1JGmPF1wjJRxY2SYsCYS7', 'input': {'query': 'weather in San Francisco'}, 'name': 'search', 'type': 'tool_use'}]
Tool Calls:
  search (toolu_01M1JGmPF1wjJRxY2SYsCYS7)
 Call ID: toolu_01M1JGmPF1wjJRxY2SYsCYS7
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: search

["I looked up: weather in San Francisco. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."]
================================== Ai Message ==================================

Based on the search results, I can provide you with information about the w

## Checking history

Let's browse the history of this thread, from start to finish.

In [8]:
all_states = []
for state in app.get_state_history(config):
    print(state)
    all_states.append(state)
    print("--")

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3103d-f96d-65f2-bfff-f32837888e44'}}, metadata={'source': 'input', 'step': -1, 'writes': {'messages': [HumanMessage(content='Use the search tool to look up the weather in SF')]}}, created_at='2024-06-23T01:56:57.764168+00:00', parent_config=None)
--
StateSnapshot(values={'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='8948e4e1-0d86-4dfd-929d-48da47fc0a18')]}, next=('agent',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3103d-f974-6b0e-8000-99038b849969'}}, metadata={'source': 'loop', 'step': 0, 'writes': None}, created_at='2024-06-23T01:56:57.767174+00:00', parent_config=None)
--
StateSnapshot(values={'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='8948e4e1-0d86-4dfd-929d-48da47fc0a18'), AIMessage(content=[{'text': "Certainly! I'll use the search tool to look up the wea

## Replay a state

We can go back to any of these states and restart the agent from there! Let's go back to right before the tool call gets executed.

In [9]:
to_replay = all_states[2]

In [10]:
to_replay.values

{'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='8948e4e1-0d86-4dfd-929d-48da47fc0a18'),
  AIMessage(content=[{'text': "Certainly! I'll use the search tool to look up the weather in San Francisco for you. Let me do that right away.", 'type': 'text'}, {'id': 'toolu_01M1JGmPF1wjJRxY2SYsCYS7', 'input': {'query': 'weather in San Francisco'}, 'name': 'search', 'type': 'tool_use'}], response_metadata={'id': 'msg_01JGRvXQBPCGbXxHHHj5mSQs', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 363, 'output_tokens': 82}}, id='run-3ec0086e-31c5-43e3-86cc-badaf14dd001-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in San Francisco'}, 'id': 'toolu_01M1JGmPF1wjJRxY2SYsCYS7'}])]}

In [11]:
to_replay.next

('action',)

To replay from this place we just need to pass its config back to the agent. Notice that it just resumes from right where it left all - making a tool call.

In [13]:
for event in app.stream(None, to_replay.config):
    for v in event.values():
        print(v)

{'messages': [ToolMessage(content='["I looked up: weather in San Francisco. Result: It\'s sunny in San Francisco, but you better look out if you\'re a Gemini 😈."]', name='search', tool_call_id='toolu_01M1JGmPF1wjJRxY2SYsCYS7')]}
{'messages': [AIMessage(content='Based on the search results, I can provide you with information about the weather in San Francisco:\n\nThe current weather in San Francisco is sunny. This means it\'s a clear day with plenty of sunshine, which is quite typical for San Francisco, especially during certain times of the year.\n\nHowever, there\'s an interesting and somewhat humorous addition to the weather report. It mentions, "but you better look out if you\'re a Gemini 😈." This appears to be a playful reference to astrology, suggesting that Geminis might have some challenges or interesting experiences today. Of course, this is not a scientific weather forecast and is likely just added for entertainment value.\n\nTo summarize:\n1. The weather in San Francisco is c

## Branch off a past state

Using LangGraph's checkpointing, you can do more than just replay past states. You can branch off previous locations to let the agent explore alternate trajectories or to let a user "version control" changes in a workflow.

Let's show how to do this to edit the state at a particular point in time. Let's update the state to change the input to the tool

In [14]:
# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values["messages"][-1]

# Let's now update the args for that tool call
last_message.tool_calls[0]["args"] = {"query": "current weather in SF"}

branch_config = app.update_state(
    to_replay.config,
    {"messages": [last_message]},
)

We can then invoke with this new `branch_config` to resume running from here with changed state. We can see from the log that the tool was called with different input.

In [17]:
for event in app.stream(None, branch_config):
    for v in event.values():
        print(v)

{'messages': [ToolMessage(content='["I looked up: current weather in SF. Result: It\'s sunny in San Francisco, but you better look out if you\'re a Gemini 😈."]', name='search', tool_call_id='toolu_01M1JGmPF1wjJRxY2SYsCYS7')]}
{'messages': [AIMessage(content='Thank you for providing me with the search results. Based on the information I received, I can tell you about the current weather in San Francisco:\n\n1. It\'s currently sunny in San Francisco.\n\nThis means you can expect clear skies and pleasant weather conditions. It\'s a great day to be outdoors or enjoy activities in the city.\n\nInterestingly, the search result also included an unusual astrological reference:\n\n2. There was a playful warning for Geminis, suggesting they should "look out."\n\nPlease note that the astrological comment is likely just for fun and not related to the actual weather conditions. It\'s probably part of a horoscope or similar content that was included in the search results.\n\nIs there anything else y

Alternatively, we could update the state to not even call a tool!

In [18]:
from langchain_core.messages import AIMessage

# Let's now get the last message in the state
# This is the one with the tool calls that we want to update
last_message = to_replay.values["messages"][-1]

# Let's now get the ID for the last message, and create a new message with that ID.
new_message = AIMessage(content="its warm!", id=last_message.id)

branch_config = app.update_state(
    to_replay.config,
    {"messages": [new_message]},
)

In [19]:
branch_state = app.get_state(branch_config)

In [20]:
branch_state.values

{'messages': [HumanMessage(content='Use the search tool to look up the weather in SF', id='8948e4e1-0d86-4dfd-929d-48da47fc0a18'),
  AIMessage(content='its warm!', id='run-3ec0086e-31c5-43e3-86cc-badaf14dd001-0')]}

In [21]:
branch_state.next

()

You can see the snapshot was updated and now correctly reflects that there is no next step.